In [1]:
import numpy as np
import pandas as pd 
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import transformers

In [2]:
torch.__version__, transformers.__version__
print(torch.cuda_version)
torch.cuda.is_available()

11.8


True

In [100]:
df_test = pd.read_csv("C:\\Users\\User\\Documents\\GitHub\\Masther_Thesis\\tweets\\stock_market_tweets_train.csv")


df_test["body"] = df_test["body"].str.replace("$AAPL", "Apple").str.replace("AAPL", "Apple").str.replace("$appl", "Apple").str.replace("appl", "Apple")
df_test['body'] = df_test['body'].apply(lambda x: x.replace("$Apple", "Apple"))

df_test["body"] = df_test["body"].str.replace("$AMZN", "Amazon").str.replace("AMZN", "Amazon").str.replace("$amzn", "Amazon").str.replace("amzn", "Amazon")
df_test['body'] = df_test['body'].apply(lambda x: x.replace("$Amazon", "Amazon"))

df_test["body"] = df_test["body"].str.replace("$GOOG", "Google").str.replace("GOOG", "Google").str.replace("$googl", "Google").str.replace("goog", "Google")
df_test['body'] = df_test['body'].apply(lambda x: x.replace("$Google", "Google"))

df_test["body"] = df_test["body"].str.replace("$FB", "Facebook").str.replace("FB", "Facebook").str.replace("$fb", "Facebook").str.replace("fb", "Facebook")
df_test['body'] = df_test['body'].apply(lambda x: x.replace("$Facebook", "Facebook"))

df_test["body"] = df_test["body"].str.replace("$MSFT", "Microsoft").str.replace("MSFT", "Microsoft").str.replace("$msft", "Microsoft").str.replace("msft", "Microsoft")
df_test['body'] = df_test['body'].apply(lambda x: x.replace("$Microsoft", "Microsoft"))

df_test["body"] = df_test["body"].str.replace("$TSLA", "Tesla").str.replace("TSLA", "Tesla").str.replace("$tsla", "Tesla").str.replace("tsla", "Tesla")
df_test['body'] = df_test['body'].apply(lambda x: x.replace("$Tesla", "Tesla"))

df_test["company"] = df_test["ticker_symbol"].str.replace("AAPL", "Apple").str.replace("AMZN", "Amazon").str.replace("GOOG", "Google").str.replace("FB", "Facebook").str.replace("MSFT", "Microsoft").str.replace("TSLA", "Tesla")

df_test["text"] = df_test.apply(lambda row: f"Talking about {row['company']}. {row['body']}", axis=1)

# df_test["text"] = df_test["body"]

df = df_test[["text", "Sentiment"]].dropna()

rename_dict = {"text": "sentence", "Sentiment": "label"}
df = df.rename(columns=rename_dict)


<ipython-input-100-fb3aaae6f4f5>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test["body"] = df_test["body"].str.replace("$AAPL", "Apple").str.replace("AAPL", "Apple").str.replace("$appl", "Apple").str.replace("appl", "Apple")
<ipython-input-100-fb3aaae6f4f5>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test["body"] = df_test["body"].str.replace("$AMZN", "Amazon").str.replace("AMZN", "Amazon").str.replace("$amzn", "Amazon").str.replace("amzn", "Amazon")
<ipython-input-100-fb3aaae6f4f5>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test["body"] = df_test["body"].str.replace("$GOOG", "Google").str.replace("GOOG", "Google").str.replace("$googl", "Google").str.replace("goog", "Google")
<ipython-input-100-fb3aaae6f4f5>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df_te

In [102]:
df['sentence']


0       Talking about Apple. Insanity of today weirdo ...
1       Talking about Apple. My biggest winner in 2014...
2       Talking about Apple. Had a down day of -.66%. ...
3       Talking about Apple. YR %,  /-, Tesla  47.85%,...
4       Talking about Google. Prediction: $TWTR $GRPN ...
                              ...                        
9995    Talking about Google. @zendecisions @Eternity1...
9996    Talking about Apple. Apple 109 calls looking l...
9997    Talking about Apple. My Weekend Review VIDEO: ...
9998    Talking about Apple. @basche42 agreed. From EC...
9999    Talking about Apple. "@Charles_Darwin: $GILD F...
Name: sentence, Length: 9776, dtype: object

In [103]:
df['label'] = df['label'].str.replace(' ', '')

df = df.dropna(subset=['label'])

# Split 'label' column into new columns
df['label'] = df['label'].str[2]

# Convert the new columns to numeric values
# df['number1'] = pd.to_numeric(df['number1'])
df['label'] = pd.to_numeric(df['label'])
df = df.dropna()

df['label'] = df['label'].astype(int)




In [104]:
# Take the smaller sample size
sample_size = df['label'].value_counts().min()

# Use resample to balance the sample
df = pd.concat([
    df[df['label'] == value].sample(sample_size, random_state=42)
    for value in df['label'].unique()
])

value_counts = df['label'].value_counts()

print(value_counts)

2    2318
0    2318
1    2318
Name: label, dtype: int64


In [105]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.22, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.21, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(4284, 2) (1530, 2) (1140, 2)


In [106]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=512), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=512), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=512), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=5,
        per_device_eval_batch_size=5,
        num_train_epochs=50,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics
)

trainer.train()   


In [ ]:
model.eval()
trainer.predict(dataset_val).metrics

In [27]:
set = trainer.predict(dataset_val)

  0%|          | 0/228 [00:00<?, ?it/s]

In [ ]:
dataset_val

In [ ]:
# Convert arrays to columns using pd.Series
expanded_columns = set['prediction'].apply(pd.Series)

# Merge the expanded columns into the dataset
dataset = dataset.add_columns(expanded_columns)

# Remove the original 'prediction' column
dataset = dataset.remove_columns('prediction')

In [35]:
test = dataset_val['sentence'][0]

In [47]:
test.replace("$Tesla", "Tesla")

'Talking about Tesla. loss on Tesla puts as well'

In [49]:
line = np.argmax(set.predictions, axis=1)

line

array([2, 1, 1, ..., 2, 1, 0], dtype=int64)

In [53]:
dataset_val_test = dataset_val.add_column('predictions', np.argmax(set.predictions, axis=1))

In [57]:
df_ev = dataset_val_test.to_pandas()

In [61]:
df_ev = df_ev[['sentence', 'label', 'predictions']]

In [ ]:
df_ev['correct'] = df_ev['label'] == df_ev['predictions']

In [65]:
df_ev

,sentence,label,predictions,correct
0,Talking about Tesla. loss on Tesla puts as well,2,2,True
1,Talking about Apple. grabbed Apple 112 calls ....,1,1,True
2,Talking about Apple. I expect $aapl to make a ...,1,1,True
3,Talking about Amazon. $Amazon is coming @howar...,2,2,True
4,Talking about Apple. Apple through 9ema resist...,0,0,True
...,...,...,...,...
1135,Talking about Apple. Apple 111.06 Over 111. Am...,1,1,True
1136,Talking about Apple. Report: Apple set to ship...,0,0,True
1137,Talking about Tesla. Tesla downgraded to $170 ...,2,2,True
1138,Talking about Apple. out remainder Apple 112 c...,1,1,True


In [70]:
from sklearn.metrics import confusion_matrix

In [71]:
results = {'TP': {}, 'FP': {}, 'FN': {}, 'TN': {}}

for value in [0, 1, 2]:
    label_values = df_ev['label'] == value
    prediction_values = df_ev['predictions'] == value

    conf_matrix = confusion_matrix(label_values, prediction_values)
    tp = conf_matrix[1, 1]
    fp = conf_matrix[0, 1]
    fn = conf_matrix[1, 0]
    tn = conf_matrix[0, 0]

    results['TP'][value] = tp
    results['FP'][value] = fp
    results['FN'][value] = fn
    results['TN'][value] = tn

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

print(results_df)

    TP   FP   FN   TN
0  267  119  113  641
1  279  111  101  649
2  271   93  109  667


In [111]:
trainer.save_model("C:\\Users\\User\\Documents\\GitHub\\Masther_Thesis\\Seccond_try")

In [74]:
incorrect_sentences = df_ev.loc[~df_ev['correct'], ['sentence', 'label', 'predictions']]

print(incorrect_sentences['sentence'][0])

                                               sentence  label  predictions
10    Talking about Tesla. Baird Equity says slowing...      1            2
14    Talking about Apple. Clearly Apple pass on EMA...      2            1
15    Talking about Apple. Have to figure some analy...      1            0
25    Talking about Apple. those 1500 contracts of t...      2            1
27    Talking about Tesla. @mansiondave STILL WAITIN...      0            1
...                                                 ...    ...          ...
1119  Talking about Apple. if you did, i would take ...      2            1
1122  Talking about Apple. New #Xiaomi phone much mo...      0            2
1124  Talking about Amazon. $Amazon $PCLN Out of my ...      0            1
1133  Talking about Apple. Apple relative strength a...      1            0
1139  Talking about Apple. "@vrdrk: Apple where's th...      1            0

[323 rows x 3 columns]


In [98]:
set.predictions[1122]

array([-4.596674 , -3.9067338,  9.029173 ], dtype=float32)

In [99]:
incorrect_sentences['sentence'][1122]

"Talking about Apple. New #Xiaomi phone much more of a threat to #Samsung than to Apple. And Samsung's #Tizen experiment won't make any difference."